# Abstracted Python Commands for Aerospike

## Prerequisites

- Create an Aerospike instance by following the installation instructions, such as running an instance on a docker container.
- Retrieve information about your host URL and the name of your collection, e.g. a multi-map.
- Install and import the library aerospike and json.

In [69]:
!pip install aerospike


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [70]:
import json
import aerospike

## Python Class for Aerospike

In [101]:
import aerospike
import json
import time

class AerospikeClient:
    def __init__(self, host, port, namespace, set_name):
        self.config = {"hosts": [(host, port)]}
        self.namespace = namespace
        self.set_name = set_name

    def connect(self):
        self.client = aerospike.client(self.config).connect()

    def disconnect(self):
        self.client.close()

    def insert_data(self, json_file):
        # Open the JSON file
        with open(json_file, "r") as file:
            json_data = json.load(file)

        # Iterate over each item in the JSON data
        for item in json_data:
            # Extract the field values
            id = item["id"]
            lclid = item["LCLid"]
            day = item["day"]
            energy_median = float(item["energy_median"])
            energy_mean = float(item["energy_mean"])
            energy_max = float(item["energy_max"])
            energy_count = float(item["energy_count"])
            energy_std = float(item["energy_std"])
            energy_sum = float(item["energy_sum"])
            energy_min = float(item["energy_min"])

            # Define the Aerospike key using the "id" field as the primary key
            key = (self.namespace, self.set_name, id)

            # Set an Aerospike bin with the bin name "energy" and put the record_dict
            bins = {
                "energy_mm": {
                    "id": id,
                    "lclid": lclid,
                    "day": day,
                    "energy_median": energy_median,
                    "energy_mean": energy_mean,
                    "energy_max": energy_max,
                    "energy_count": energy_count,
                    "energy_std": energy_std,
                    "energy_sum": energy_sum,
                    "energy_min": energy_min,
                }
            }

            # Create the write policy
            write_policy = {"key": aerospike.POLICY_KEY_SEND}

            # Write the record to Aerospike
            try:
                self.client.put(key=key, bins=bins, policy=write_policy)
            except aerospike.exception.AerospikeError as e:
                print(f"Create failed for Record with id: {id}\nError: {e.msg}, {e.code}")

    def query_data(self, primary_keys):
        for pk in primary_keys:
            key = (self.namespace, self.set_name, pk)
            try:
                _, _, record = self.client.get(key)
                energy = record["energy_mm"]
                print(f"PK: {pk}, Energy: {energy}")
            except aerospike.exception.RecordNotFound:
                print(f"No record found for PK: {pk}")

## Exemplary Usage

In [107]:
aerospike_client = AerospikeClient("localhost", 3000, "test", "energy_mm")
aerospike_client.connect()

In [ ]:
aerospike_client.insert_data("/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/3_KVS/CosmosDB/iter2.2/daily_dataset_small_kv.json")

In [ ]:
record = aerospike_client.query_data("MAC000131_2011-12-15")
print(record)

# Abstracted Python Commands for CosmosDB

## Prerequisites

- Create a CosmosDB account in Microsoft Azure.
- Retrieve information about your endpoint and your key to establish a client to CosmosDB.
- Retrieve information about your database name and container name you have created via the web interface in your CosmosDB account in Azure.
- Install the library azure-cosmos and import the libraries CosmosClient and json.

In [1]:
!pip install azure-cosmos


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [14]:
import json
from azure.cosmos import CosmosClient

## Python Class for CosmosDB

In [66]:
class CosmosDBClient:
    def __init__(self, endpoint, key, database_name, container_name):
        self.client = CosmosClient(endpoint, key)
        self.database_name = database_name
        self.container_name = container_name
        
    def disconnect(self):
        self.client.close()
    
    def insert_data(self, json_file, container_item):
        """Specify the path to your json file with json_file and the item names eventually named in the container with container_item"""
        database = self.client.get_database_client(self.database_name)
        container = database.get_container_client(self.container_name)

        with open(json_file, "r") as file:
            json_data = json.load(file)

        for item in json_data:
            item_data = {key: item[key] for key in container_item}
            item_data["partitionKey"] = "default"

            container.create_item(item_data)

    def query_data(self, query):
        """Specify your query and pass them into this function."""
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        return items

## Exemplary Usage

In [67]:
endpoint = "https://smart-meter-benchmark.documents.azure.com:443/"
key = "HvrYXT5hnZmhRNJf2vJdXRy7Ms4YSKQE6lP77MAw4AES7N7sXLopbTyy1LoXYeBwRLgKEGms3S0JACDbKq9F3Q==;"
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"

client = CosmosDBClient(endpoint, key, database_name, container_name)

In [62]:
json_file = "/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/3_KVS/CosmosDB/iter2.2/daily_dataset_small_kv.json"

container_item = [
    "id",
    "LCLid",
    "day",
    "energy_median",
    "energy_mean",
    "energy_max",
    "energy_count",
    "energy_std",
    "energy_sum",
    "energy_min"
]


client.insert_data(json_file=json_file, container_item=container_item)

In [68]:
query = "SELECT * FROM c WHERE c.id = 'MAC000131_2011-12-15'"

client.query_data(query)

[{'id': 'MAC000131_2011-12-15',
  'LCLid': 'MAC000131',
  'day': '2011-12-15',
  'energy_median': 0.485,
  'energy_mean': 0.4320454545454545,
  'energy_max': 0.868,
  'energy_count': 22,
  'energy_std': 0.2391457967876753,
  'energy_sum': 9.505,
  'energy_min': 0.072,
  'partitionKey': 'default',
  '_rid': 'C68OAMGhsOYBAAAAAAAAAA==',
  '_self': 'dbs/C68OAA==/colls/C68OAMGhsOY=/docs/C68OAMGhsOYBAAAAAAAAAA==/',
  '_etag': '"02002a2c-0000-1000-0000-64b7e4f80000"',
  '_attachments': 'attachments/',
  '_ts': 1689773304}]

# Abstracted Python Commands for Hazelcast

## Prerequisites

- Create a Hazelcast instance following the instructions for installation.
- Retrieve information about the URL to your host and the name of your collection, such as your multimap.
- Install the library hazelcast and import it as well as the json library.

In [1]:
!pip install hazelcast

In [2]:
import hazelcast
import json

## Python Class for Hazelcast

In [137]:
class HazelcastClient:
    def __init__(self, cluster_members, multi_map_name):
        self.client = hazelcast.HazelcastClient(cluster_members)
        self.multi_map = self.client.get_multi_map(multi_map_name)
        
    def disconnect(self):
        self.client.shutdown()

    def insert_data(self, json_file, multi_map_name):
        with open(json_file, 'r') as file:
            data = json.load(file)
            
        multi_map = self.client.get_multi_map(multi_map_name)

        for item in data:
            key = item['id']
            values = [
                item['LCLid'],
                item['day'],
                item['energy_median'],
                item['energy_mean'],
                item['energy_max'],
                item['energy_count'],
                item['energy_std'],
                item['energy_sum'],
                item['energy_min']
            ]
            multi_map.put_all({key: values})
            
    def query_values(self, keys):
        for key in keys:
            values_future = self.multi_map.get(key)
            values = values_future.result()
            print(f"Values for key '{key}': {values}")

## Exemplary Usage

In [ ]:
cluster_members = ["127.0.0.1:5701", "127.0.0.1:5702"]
multi_map_name = "smart-meter-benchmark_mm"
hazelcast_client = HazelcastClient(cluster_members, multi_map_name)

In [ ]:
hazelcast_client.insert_data('/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/3_KVS/Hazelcast/iter2.2/daily_dataset_small_kv.json', "smart-meter-benchmark_mm")

In [ ]:
keys = ["MAC000131_2012-06-01", "MAC000131_2012-06-02", "MAC000131_2012-06-03"]
hazelcast_client.query_values(keys)